In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# use tex
plt.rc("text", usetex=True)

In [ ]:
c_df = pd.read_csv("data/c.csv", comment="#")
c_df

In [ ]:
rho_df = pd.read_csv("data/rho.csv", comment="#")
rho_df

In [ ]:
au_df = pd.read_csv("data/au.csv", comment="#")
au_df

In [ ]:
avogadro_df = pd.read_csv("data/bailey/CsvData/Constants_Data/Avogadro_Number.csv")
# remove last row
avogadro_df = avogadro_df.iloc[:-1]
avogadro_df

In [ ]:
finestructure_df = pd.read_csv(
    "data/bailey/CsvData/Constants_Data/Fine_Structure_Constant_Inverse.csv"
)
finestructure_df

In [ ]:
rydberg_df = pd.read_csv("data/bailey/CsvData/Constants_Data/Rydberg_Constant.csv")
rydberg_df

In [ ]:
c_bailey_df = pd.read_csv("data/bailey/CsvData/Constants_Data/Speed_of_Light.csv")
c_bailey_df

In [ ]:
datasets = {
    "rho": rho_df,
    "c": c_df,
    "au": au_df,
    "avogadro": avogadro_df,
    "finestructure": finestructure_df,
    "rydberg": rydberg_df,
    # 'c_bailey': c_bailey_df,
}
truths = {
    "rho": 5.513,
    "c": 299792.458,
    "au": 149597870700,
    "avogadro": 6.02214076e23,
    "finestructure": [137.035999177, 0.000000021],
    "rydberg": [10973731.568157, 0.000012],
    "c_bailey": 299792.458,
}
yscales = {
    "rho": "symlog",
    "c": "symlog",
    "au": "symlog",
    "avogadro": "symlog",
    "finestructure": "symlog",
    "rydberg": "symlog",
    # 'c_bailey': 'symlog',
}
linthresh = {
    "rho": 0.01,
    "c": 0.1,
    "au": 1,
    "avogadro": 1e17,
    "finestructure": 1e-6,
    "rydberg": 1e-4,
    # 'c_bailey': 0.1,
}

nice_names = {
    "c": "Speed of light",
    "rho": "Density of Earth",
    "au": "Astronomical Unit",
    "avogadro": "Avogadro Number",
    "finestructure": "Inverse Fine Structure Constant",
    "rydberg": "Rydberg Constant",
    # 'c_bailey': 'Speed of light (bailey)',
}

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(8, 8), sharey=True, gridspec_kw={"wspace": 0.05})
xlabels = {
    "rho": r"Difference from true value $[\mathrm{g/cm^3}]$",
    "c": r"Difference from true value $[\mathrm{km/s}]$",
    "au": r"Difference from true value $[\mathrm{km}]$",
    "avogadro": r"Difference from true value $[\mathrm{mol^{-1}}]$",
    "finestructure": r"Difference from true value",
    "rydberg": r"Difference from true value $[\mathrm{m^{-1}}]$",
    # 'c_bailey': r'Difference from true value $[\mathrm{km/s}]$',
}
historical_keys = [
    "rho",
    "c",
    "au",
    "avogadro",
    "finestructure",
    "rydberg",
]  # , 'c_bailey']
for i, name in enumerate(historical_keys):
    ax = axs.flatten()[i]
    if hasattr(truths[name], "__iter__"):
        truth = truths[name][0]
    else:
        truth = truths[name]

    values = datasets[name].value - truth
    if name == "au":
        values = values / 1000
    dates = datasets[name].year

    ax.plot(values, dates, ".", color="black")
    ax.axvline(0, color="black", linestyle="--", linewidth=1)
    # reverse y axis
    ax.invert_yaxis()

    if yscales[name] == "symlog":
        print(name)
        ax.set_xscale("symlog", linthresh=linthresh[name])
        # skip every other tick
        n_ticklabels = len(ax.xaxis.get_ticklabels())
        for n, label in enumerate(ax.xaxis.get_ticklabels()):
            if n % 2 != 0 and label.get_text() != "$\\mathdefault{0}$":
                label.set_visible(False)
        ax.tick_params(axis="both", which="both", direction="in", top=True, right=True)
    ax.set_xlabel(xlabels[name])
    ax.set_ylim(2015, 1650)
    ax.set_title(nice_names[name])
    # make top x limit and bottom x limit equal
    xlim = max(abs(ax.get_xlim()[0]), abs(ax.get_xlim()[1]))
    ax.set_xlim(-xlim, xlim)
# axs[0].set_ylabel('Year')

plt.tight_layout()
plt.savefig("figs/historical.pdf", bbox_inches="tight")
plt.show()

In [ ]:
# get error on ratio calculations
def clarke_ratio(A, a, B, b, C=100, c=0):
    # if A:B is the ratio, we want x where A:B = C:x
    value = (B / A) * C
    uncertainty = np.sqrt((B * C * a / A) ** 2 + (C * b) ** 2 + (B * c) ** 2) / A
    return value, uncertainty


def clarke_quotient(A, a, B, b):
    # get the ratio B:A and its error
    value = B / A
    uncertainity = np.sqrt((B * a / A) ** 2 + b**2) / A
    return value, uncertainity

In [ ]:
ho_df = pd.read_csv("data/clarke/H-O-mass.csv", comment="#")
ho_df["uncertainty"] = ho_df["proberr"] / 0.6745
agcl_df = pd.read_csv("data/clarke/Ag-Cl-mass.csv", comment="#")
agcl_df["uncertainty"] = agcl_df["proberr"] / 0.6745
agi_df = pd.read_csv("data/clarke/Ag-I-mass.csv", comment="#")
agi_df["uncertainty"] = agi_df["proberr"] / 0.6745
agbr_df = pd.read_csv("data/clarke/Ag-Br-mass.csv", comment="#")
agbr_df["uncertainty"] = agbr_df["proberr"] / 0.6745
no_df = pd.read_csv("data/clarke/N-mass.csv", comment="#")
no_df["uncertainty"] = no_df["proberr"] / 0.6745
co_df = pd.read_csv("data/clarke/C-mass.csv", comment="#")
co_df["uncertainty"] = co_df["proberr"] / 0.6745

datasets["ho"] = ho_df
datasets["agcl"] = agcl_df
datasets["agi"] = agi_df
datasets["agbr"] = agbr_df
# datasets['no'] = no_df
# datasets['co'] = co_df

truths["agcl"] = clarke_ratio(107.8682, 0.0002, 35.453, 0.004, 100, 0)
truths["agbr"] = clarke_ratio(107.8682, 0.0002, 79.904, 0.003, 100, 0)
truths["agi"] = clarke_ratio(107.8682, 0.0002, 126.90447, 0.00003, 100, 0)
truths["ho"] = clarke_quotient(1.0080, 0.0002, 15.9995, 0.0005)
# truths['no'] = clarke_ratio(15.9995, 0.0005, 14.007, 0.001, 16, 0)
# truths['co'] = clarke_ratio(15.9995, 0.0005, 12.011, 0.002, 16, 0)

nice_names["ho"] = "O:H mass ratio"
nice_names["agcl"] = "Ag:Cl mass ratio"
nice_names["agi"] = "Ag:I mass ratio"
nice_names["agbr"] = "Ag:Br mass ratio"
# nice_names['no'] = 'N mass (O=16)'
# nice_names['co'] = 'C mass (O=16)'

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(8, 3))
from methods import birge, random_effects_hksj, binomial_method

plot_est = False
chemistry_keys = ["ho", "agcl", "agi", "agbr"]

for i, name in enumerate(chemistry_keys):
    ax = axs.flatten()[i]
    # values = np.array(datasets[name].value - truths[name])
    truth = truths[name][0]
    err = truths[name][1]
    ax.axvspan(truth - err, truth + err, color="black", alpha=0.3)

    values = np.array(datasets[name].value)
    errs = np.array(datasets[name].uncertainty)
    # sort by decreasing error
    sort_idx = np.argsort(errs)[::-1]
    values = values[sort_idx]
    errs = errs[sort_idx]

    ax.errorbar(
        values,
        np.arange(len(values)),
        xerr=errs,
        fmt=".",
        markersize=2,
        linewidth=1,
        color="black",
    )
    ax.invert_yaxis()
    # xlim = max(abs(ax.get_xlim()[0]), abs(ax.get_xlim()[1]))
    # ax.set_xlim(-xlim, xlim)
    if plot_est:
        interval_birge, _, _, _ = birge(values, errs, coverage=0.6827)
        ax.axvline(interval_birge[0], color="red", linestyle="--", linewidth=1)
        ax.axvline(interval_birge[1], color="red", linestyle="--", linewidth=1)
        interval_re, _, _, _ = random_effects_hksj(values, errs, coverage=0.6827)
        ax.axvline(interval_re[0], color="blue", linestyle="--", linewidth=1)
        ax.axvline(interval_re[1], color="blue", linestyle="--", linewidth=1)
        binomial_lower, _ = binomial_method(
            values, p=0.5, target=0.15865, which="lower"
        )
        ax.axvline(binomial_lower, color="green", linestyle="--", linewidth=1)
        binomial_upper, _ = binomial_method(
            values, p=0.5, target=0.15865, which="upper"
        )
        ax.axvline(binomial_upper, color="green", linestyle="--", linewidth=1)

    ax.set_title(nice_names[name])

    # ax.axvline(truth, color='black', linestyle='--', linewidth=1)
    # ax.axvline(truth-err, color='grey', linestyle='--', linewidth=1)
    # ax.axvline(truth+err, color='grey', linestyle='--', linewidth=1)

    # ax.set_xlabel(xlabels[name])
    # remove y ticks
    ax.set_yticks([])
    ax.get_xaxis().get_major_formatter().set_useOffset(False)
    xmin = ax.get_xlim()[0]
    xmax = ax.get_xlim()[1]
    ax.set_xlim(min(xmin, truth - err), max(xmax, truth + err))
    # add top ticks
    ax.tick_params(axis="x", top=True)
    # make ticks point inwards
    ax.tick_params(direction="in")
plt.tight_layout()
plt.savefig("figs/chemical.pdf", bbox_inches="tight")
plt.show()

In [ ]:
import os

particle_keys = sorted(
    [f.split(".")[0] for f in os.listdir("data/pdg1970") if f.endswith(".csv")]
)
print(particle_keys)
units = {}
for p in particle_keys:
    path = f"data/pdg1970/{p}.csv"
    datasets[p] = pd.read_csv(path, comment="#")
    with open(path, "r") as f:
        lines = f.readlines()
        nice_names[p] = lines[0].strip("# ").strip("\n")
        units[p] = lines[1].strip("# ").strip("\n")
        value = float(lines[2].split(":")[1].strip())
        sigma = float(lines[3].split(":")[1].strip())
        truths[p] = (value, sigma)
print(len(particle_keys))

In [ ]:
particle_group_keys = {
    "T": "Lifetime",
    "MM": "Mag.~moment",
    "M": "Mass",
    "M-": "Mass",
    "DEL": "Decay param",
    "D": "Mass diff",
    "WR": "Branching rate",
    "W": "Width",
    "R": "Branching ratio",
    "L+E": "Decay param",
    "A": "Decay param",
    "F+-": "Decay param",
    "XI": "Decay param",
}
from collections import defaultdict

particle_group_map = {}
remaining = set(particle_keys)
for k, v in particle_group_keys.items():
    for p in remaining:
        if k in p[1:]:
            particle_group_map[p] = v
            remaining = remaining - set([p])
assert len(remaining) == 0, print(remaining)
nice_names.update({v: v for v in particle_group_keys.values()})

In [ ]:
from scipy.stats import binomtest
from methods import sign_rank_test

names = list(datasets.keys())

results = {}


def format_number(x):
    if x == -np.inf:
        return r"$\approx-\infty$"
    if x == np.inf:
        return r"$\approx\infty$"
    if x >= 1e2 or x <= -1e2:
        return f"${str(int(x))}$"
    return r"$\num{{{0:.2g}}}$".format(x)


def fmt_result(result):
    # result is a list
    minmax = [min(result), max(result)]
    result_fmt = [format_number(x) for x in minmax]
    # get unique values (preserving order)
    result_fmt = pd.unique(np.array(result_fmt))
    if len(result_fmt) == 1:
        return result_fmt[0]
    else:
        return f'[{", ".join(result_fmt)}]'


for n in names:
    results[n] = {}
    results[n]["count"] = [len(datasets[n])]

    truth_vals = []
    if hasattr(truths[n], "__iter__"):
        truth_vals.append(truths[n][0] - truths[n][1])
        truth_vals.append(truths[n][0] + truths[n][1])
        truth_vals.append(truths[n][0])
    else:
        truth_vals.append(truths[n])

    results[n]["num_over"] = [np.sum(datasets[n].value > t) for t in truth_vals]
    results[n]["prop_over"] = [
        np.sum(datasets[n].value > t) / len(datasets[n]) for t in truth_vals
    ]

    # binomial test
    results[n]["binom_p_value"] = [
        binomtest(
            np.sum(datasets[n].value > t),
            len(datasets[n]),
            p=0.5,
            alternative="two-sided",
        ).pvalue
        for t in truth_vals
    ]

    results[n]["sign_rank_p_value"] = [
        sign_rank_test(datasets[n].value, t) for t in truth_vals
    ]

results_tex = {}
for n, res in results.items():
    results_tex[n] = {}
    for k, val in res.items():
        results_tex[n][k] = fmt_result(val)

# put all the results in a pandas dataframe
results_df = pd.DataFrame(results_tex).T
results_df = results_df.drop(index=particle_keys)
latex_names = [nice_names[k] for k in results_df.index]


# use np to save latex table with & separator
rows = results_df.values.tolist()
rows = [list(row) for row in rows]
rows = [[latex_names[i]] + rows[i] for i in range(len(rows))]
txt = " \\\\\n".join([" & ".join(row) for row in rows])
print(txt)
with open("tables/hist-sym.tex", "w") as f:
    f.write(txt)

In [ ]:
print(len(particle_keys))
from methods import birge, random_effects_hksj, sign_rank

# particle_keys = ['lambda-lifetime', 'sigma+-lifetime', 'pion-mass-diff', 'charged-kaon-lifetime', 'charged-pion-lifetime', 'eta-mass']


def pval_to_stars(pval):
    if pval < 0.001:
        return "***"
    elif pval < 0.01:
        return "**"
    elif pval < 0.05:
        return "*"


for plot in ["model", "inference"]:
    fig, axs = plt.subplots(
        8, 10, figsize=(8, 8), gridspec_kw={"wspace": 0, "hspace": 0.4}
    )
    assert len(particle_keys) <= len(axs.flatten())
    if plot == "inference":
        num_covs = defaultdict(int)
    for i, name in enumerate(particle_keys):
        ax = axs.flatten()[i]
        # values = np.array(datasets[name].value - truths[name])
        truth = truths[name][0]
        err = truths[name][1]

        values = np.array(datasets[name].value)
        errs = np.array(datasets[name].uncertainty)
        # sort by decreasing error
        sort_idx = np.argsort(datasets[name].year)
        values = values[sort_idx]
        errs = errs[sort_idx]

        ax.axvspan(truth - err, truth + err, color="black", alpha=0.3)

        ax.errorbar(
            values,
            -np.arange(len(values)),
            xerr=errs,
            fmt=".",
            markersize=2,
            linewidth=1,
            color="black",
        )
        # ax.invert_yaxis()
        # xlim = max(abs(ax.get_xlim()[0]), abs(ax.get_xlim()[1]))
        # ax.set_xlim(-xlim, xlim)

        # ax.set_title(f'{nice_names[name]} {units[name]}')
        ax.set_title(r"\texttt{" + name + "}", pad=2, fontsize=10)

        # ax.axvline(truth, color='black', linestyle='--', linewidth=1)
        # ax.axvline(truth-err, color='grey', linestyle='--', linewidth=1)
        # ax.axvline(truth+err, color='grey', linestyle='--', linewidth=1)

        # ax.set_xlabel(xlabels[name])
        # remove y ticks
        ax.set_yticks([])
        ax.set_xticks([])
        ax.get_xaxis().get_major_formatter().set_useOffset(False)

        xmin, xmax = ax.get_xlim()
        ax.set_xlim(min(xmin, truth - err), max(xmax, truth + err))
        # add top ticks
        ax.tick_params(axis="x", top=True)
        # make ticks point inwards
        ax.tick_params(direction="in")
        ymin, ymax = ax.get_ylim()
        yspan = ymax - ymin

        if plot == "model":
            pvals = [
                min(results[name]["binom_p_value"]),
                min(results[name]["sign_rank_p_value"]),
            ]
            for i, pval in enumerate(pvals):
                if pval < 0.05:
                    stars = pval_to_stars(pval)
                    side = "left" if i == 0 else "right"
                    x = 0.1 if i == 0 else 0.91
                    ax.text(
                        x,
                        -0.02,
                        stars,
                        color="black",
                        ha=side,
                        va="top",
                        transform=ax.transAxes,
                    )

        if plot == "inference":
            interval_signrank, cov = sign_rank(values, coverage=0.6827)

            interval_birge, _, _, _ = birge(values, errs, coverage=cov, pdg=True)
            # ax.axvline(interval_birge[0], color='red', linestyle='--', linewidth=1)
            # ax.axvline(interval_birge[1], color='red', linestyle='--', linewidth=1)
            interval_re, _, _, _ = random_effects_hksj(values, errs, coverage=cov)
            # ax.axvline(interval_re[0], color='blue', linestyle='--', linewidth=1)
            # ax.axvline(interval_re[1], color='blue', linestyle='--', linewidth=1)

            # ax.axvline(interval_signrank[0], color='green', linestyle='--', linewidth=1)
            # ax.axvline(interval_signrank[1], color='green', linestyle='--', linewidth=1)
            intervals = {
                "pdg": interval_birge,
                "re_hksj": interval_re,
                "signrank": interval_signrank,
            }

            for j, (name, interval) in enumerate(intervals.items()):
                # ax.axvspan(interval[0], interval[1])
                covers = (interval[0] <= truth + 2 * err) & (
                    truth - 2 * err <= interval[1]
                )
                color = "green" if covers else "red"
                y0 = ymin + j * yspan / 3
                y1 = ymin + (j + 1) * yspan / 3
                x0 = interval[0]
                x1 = interval[1]
                # plot a rectangle
                ax.add_patch(
                    plt.Rectangle(
                        (x0, y0), x1 - x0, y1 - y0, color=color, alpha=1, zorder=-1
                    )
                )
                # ax.axhspan(y0, y1, xmin=x0, xmax=x1, color=color, alpha=0.3, zorder=-1)
                num_covs[name] += covers
            # print(num_covs)

    # remove unused axes
    for ax in axs.flatten()[len(particle_keys) :]:
        ax.set_visible(False)

    # plt.tight_layout()
    filename = "particles.pdf" if plot == "model" else "particles-inference.pdf"
    plt.savefig(f"figs/{filename}", bbox_inches="tight")
    plt.show()
print(num_covs)

In [ ]:
from scipy.optimize import minimize
from scipy.stats import chi2

# print(values)
# res.fun
chi2.ppf(0.05, df=1)

In [ ]:
from methods import birge, random_effects_mle
from scipy.stats import norm, chi2
from collections import defaultdict
from tqdm import tqdm

results = {}


def fmt_result(r, bold=False):
    if bold:
        start = r"$\mathbf{"
        end = r"}$"
    else:
        start = r"$"
        end = r"$"
    if r == -np.inf:
        return r"$\approx-\infty$"
    elif r < -100:
        return start + str(int(r)) + end
    else:
        return start + r"{:.3g}".format(r) + end


category_map = {k: "chemistry" for k in chemistry_keys}
category_map.update({k: "particle" for k in particle_keys})
category_map.update({k: "historical" for k in historical_keys})

key_order = []

exponent_grid = np.linspace(0, 1, 201)
for n in tqdm(names):
    values = np.array(datasets[n].value)
    sigmas = np.array(datasets[n].uncertainty)
    has_sigma = ~np.isnan(sigmas)
    values = values[has_sigma]
    sigmas = sigmas[has_sigma]
    if len(values) < 2:
        print(f"{n} has less than 2 data points")
        continue

    truth = truths[n][0] if hasattr(truths[n], "__iter__") else truths[n]
    # scaler = truth
    # values = values/scaler
    # sigmas = sigmas/scaler
    # truth = truth/scaler

    _, muhat_birge, _, chat = birge(
        values, sigmas, coverage=0.6827, truth=truth, mle=True
    )
    assert np.isclose(truth, muhat_birge)
    # brs.append(chat)
    # mean_sigma = np.mean(sigmas)
    # muhat_re, _, tau = random_effects_dl_base(values, sigmas)
    # taus.append(np.mean(tau/sigmas))
    _, muhat_re, _, tau = random_effects_mle(
        values, sigmas, coverage=0.6827, truth=truth
    )
    assert np.isclose(truth, muhat_re)
    # taus.append(np.mean(tau/sigmas))
    # I2s.append(I2(values, sigmas))

    # # generate values with same sigmas but no unaccounted for errors.
    # # to be used as a control when analyzing the distribution of chat and tau
    # values_control = np.random.normal(loc=0, scale=sigmas)
    # _, _, _, chat_cont = birge(values_control, sigmas, coverage=0.6827)
    # brs_cont.append(chat_cont)
    # _, _, _, tau_cont = random_effects_mle(values_control, sigmas, coverage=0.6827)
    # taus_cont.append(np.mean(tau_cont/sigmas))
    # I2s_cont.append(I2(values_control, sigmas))

    # # errscale_ps.append(errscale_test(values, sigmas))
    # # errscale_ps_cont.append(errscale_test(values_control, sigmas))
    if n in particle_keys:
        key = particle_group_map[n]
    else:
        key = n
    if key not in key_order:
        key_order.append(key)
    if key not in results:
        results[key] = {}
        results[key]["name"] = nice_names[key]
        results[key]["ds_count"] = 0
        results[key]["count"] = 0
        for r in ["birge_loglike", "re_loglike", "fe_loglike"]:
            results[key][r] = 0
        results[key]["exponent_loglike"] = np.zeros_like(exponent_grid)
        results[key]["exponent_loglike_iter"] = np.zeros_like(exponent_grid)
    results[key]["ds_count"] += 1
    results[key]["count"] += len(values)
    # print(norm.pdf(values, loc=muhat_birge, scale=sigmas*chat))
    results[key]["birge_loglike"] += np.sum(
        norm.logpdf(values, loc=muhat_birge, scale=sigmas * chat)
    )
    results[key]["re_loglike"] += np.sum(
        norm.logpdf(values, loc=muhat_re, scale=np.sqrt(sigmas**2 + tau**2))
    )
    results[key]["fe_loglike"] += np.sum(
        norm.logpdf(values, loc=muhat_birge, scale=sigmas)
    )

    # def to_minimize(params):
    #     tau, exponent = params
    #     return -np.sum(norm.logpdf(values, loc=truth, scale=np.sqrt(sigmas**2+(tau**2)*(sigmas**(2*exponent)))))
    # res = minimize(to_minimize, x0=(0,0), bounds=[(0,max(sigmas)*100),(0,4)])
    def to_minimize(params, exponent):
        tau = params[0]
        return -np.sum(
            norm.logpdf(
                values,
                loc=truth,
                scale=np.sqrt(sigmas**2 + (tau**2) * (sigmas ** (2 * exponent))),
            )
        )

    for i, exponent in enumerate(exponent_grid):
        res = minimize(
            to_minimize,
            x0=(0,),
            bounds=[(0, max(sigmas) * 100)],
            args=(exponent,),
            method="Nelder-Mead",
        )
        if not res.success:
            print(res)
            raise (ValueError())
        results[key]["exponent_loglike_iter"][i] += -res.fun

    #     _, muhat_re, _, tau = random_effects_mle(values, sigmas, coverage=0.6827, truth=truth, exponent=exponent)
    #     loglike = np.sum(norm.logpdf(values, loc=muhat_re, scale=np.sqrt(sigmas**2+(tau**2)*sigmas**exponent)))
    #     results[key]['exponent_loglike_iter'][i] += loglike
    tau_grid = np.concatenate(
        (np.logspace(-4, 4, 1000) * max(sigmas), np.logspace(-4, 4, 1000))
    )
    for i, exponent in enumerate(exponent_grid):
        scale = np.sqrt(
            sigmas[:, np.newaxis] ** 2
            + ((tau_grid) ** 2) * (sigmas[:, np.newaxis] ** (2 * exponent))
        )
        loglike = norm.logpdf(values[:, np.newaxis], loc=muhat_re, scale=scale)
        results[key]["exponent_loglike"][i] += np.max(np.sum(loglike, axis=0))

    results[key]["category"] = category_map[n]

for k, res in list(results.items()):
    category = res["category"]
    if category not in results:
        # dict with default value of zero
        results[category] = defaultdict(float)
    if "total" not in results:
        results["total"] = defaultdict(float)
    for r in [
        "ds_count",
        "count",
        "birge_loglike",
        "re_loglike",
        "fe_loglike",
        "exponent_loglike",
        "exponent_loglike_iter",
    ]:
        results[category][r] += res[r]
        results["total"][r] += res[r]

for res in results.values():
    res["ds_count"] = fmt_result(res["ds_count"])
    res["count"] = fmt_result(res["count"])
    if len(np.unique(res["exponent_loglike"])) == 1:
        print("all same loglike")
        res["best_exponent"] = "N/A"
    else:
        best_exponent = exponent_grid[np.argmax(res["exponent_loglike"])]
        res["best_exponent"] = fmt_result(best_exponent)
        logratio = 2 * (np.max(res["exponent_loglike"]) - res["exponent_loglike"])
        within = logratio < chi2.ppf(0.95, df=1)
        idx_l = np.argmax(within)
        idx_u = -np.argmax(within[::-1]) - 1
        low = exponent_grid[idx_l]
        high = exponent_grid[idx_u]
        res["exponent_interval"] = f"$[{np.round(low,2)}, {np.round(high,2)}]$"

    loglike_keys = ["birge_loglike", "re_loglike", "fe_loglike"]
    loglike_values = [res[k] for k in loglike_keys]
    largest_idx = np.argmax(loglike_values)
    if len(np.unique(loglike_values)) == 1:
        largest_idx = np.nan
    for i, r in enumerate(loglike_keys):
        res[r] = fmt_result(res[r], bold=(i == largest_idx))

results["total"]["name"] = "Total"
results["historical"]["name"] = "Total (historical)"
results["chemistry"]["name"] = "Total (chemistry)"
results["particle"]["name"] = "Total (particle)"
print(results)

results_df = pd.DataFrame(results).T
del results_df["category"]
del results_df["exponent_loglike"]
del results_df["exponent_loglike_iter"]
# results_df.index = [r['name'] for r in results.values()]
# sort to be: chemistry, particle, total chemistry, total particle, total
row_order = key_order + ["historical", "chemistry", "particle", "total"]
results_df = results_df.loc[row_order]
# print(results_df)

# use np to save latex table with & separator
rows = results_df.values.tolist()
hline_idxs = [5, 5 + len(chemistry_keys), -4, -4, -1]
for i in hline_idxs:
    rows[i][0] = r"\hline " + rows[i][0]
print(rows)
txt = " \\\\\n".join([" & ".join(row) for row in rows])
print(txt)
with open("tables/hist-syst.tex", "w") as f:
    f.write(txt)

In [ ]:
from measurement_dist import measurement_dist

group_keys = {
    "historical": historical_keys,
    "chemical": chemistry_keys,
    "particle": particle_keys,
}
name_map = {"historical": "Historical", "chemical": "Chemical", "particle": "PDG 1970"}
import json

for name, keys in group_keys.items():
    dfs = []
    group_truths = []
    group_truth_sigmas = []
    for key in keys:
        df = datasets[key]
        df = df[~pd.isna(df["uncertainty"])]
        if len(df) < 3:
            continue
        dfs.append(df)
        if hasattr(truths[key], "__iter__"):
            truth = truths[key][0]
            truth_sigma = truths[key][1]
        else:
            truth = truths[key]
            truth_sigma = 0
        group_truths.append(truth)
        group_truth_sigmas.append(truth_sigma)
    output = measurement_dist(dfs, group_truths, group_truth_sigmas, lists=True)
    output["name"] = name_map[name]
    with open(f"results/measurement_dist/{name}.json", "w") as f:
        json.dump(output, f)
    plt.plot(output["zspace"], output["hstar"], label=name)
    plt.plot(output["zspace"], output["hprime"], label=name)
plt.yscale("log")
plt.legend()

# Independence

In [ ]:
# measure the empirical probability of the next observation being on the same side of the ground truth as the current one.
n_pairs = np.zeros(1000, dtype=int)
same_side = np.zeros((1000, 3), dtype=int)
for i in range(1000):
    idxs = np.random.choice(len(particle_keys), size=60, replace=False)
    for idx in idxs:
        p = particle_keys[idx]
        truth = truths[p][0]
        truth_l = truths[p][0] - 1 * truths[p][1]
        truth_u = truths[p][0] + 1 * truths[p][1]
        df = datasets[p].sort_index()
        sort_idx = np.argsort(datasets[p].year + np.random.rand(len(df)) * 0.1)
        values = np.array(datasets[p]["value"])[sort_idx]

        overs = np.array([values > truth, values > truth_l, values > truth_u])
        # over = values > truth
        xors = np.array([np.bitwise_xor(over[:-1], over[1:]) for over in overs])
        n_pairs[i] += len(xors[0])
        same_sides = np.sum(~xors, axis=1)
        # print(same_sides)
        same_side[i, 0] += same_sides[0]
        same_side[i, 1] += min(same_sides)
        same_side[i, 2] += max(same_sides)
        # same_side[0] += np.sum(~xor)

In [ ]:
# print(same_side)
# print(n_pairs)
from scipy.stats import binom

p_mid = 1 - binom.cdf(k=same_side[:, 0], n=n_pairs, p=0.5)
p_generous = 1 - binom.cdf(k=same_side[:, 1], n=n_pairs, p=0.5)

In [ ]:
from scipy.stats import binomtest

intervals_mid = np.array(
    [
        list(
            binomtest(
                k=same_side[i, 0], n=n_pairs[i], p=0.5, alternative="two-sided"
            ).proportion_ci(confidence_level=0.90, method="exact")
        )
        for i in range(same_side.shape[0])
    ]
)
intervals_generous = np.array(
    [
        list(
            binomtest(
                k=same_side[i, 1], n=n_pairs[i], p=0.5, alternative="two-sided"
            ).proportion_ci(confidence_level=0.90, method="exact")
        )
        for i in range(same_side.shape[0])
    ]
)

import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 2, figsize=(8, 3))

axs[0].hist(
    p_mid,
    bins=np.linspace(0, 0.1, 21),
    histtype="step",
    color="grey",
    density=True,
    linewidth=2,
    label=r"$\theta$ as truth",
)
axs[0].hist(
    p_generous,
    bins=np.linspace(0, 0.1, 21),
    histtype="step",
    color="black",
    density=True,
    linewidth=2,
    label=r"$\theta$, $\theta-\sigma$, or $\theta+\sigma$ as truth",
)
axs[0].set_xlabel("$p$-value on $H_0$: $p=0.5$, $H_1$: $p>0.5$")
axs[0].set_ylabel("Density")
axs[0].set_xlim(0, 0.1)
axs[0].legend(frameon=False)

# histbins = np.linspace(0.45, 0.8, 41)
# axs[1].hist(intervals_mid[:,0], bins=histbins, histtype='step', color='grey', density=True, linewidth=2)
# axs[1].hist(intervals_mid[:,1], bins=histbins, histtype='step', color='grey', density=True, linewidth=2)

colors = ["grey", "black"]
for i, intervals in enumerate([intervals_mid, intervals_generous]):
    midpoints = np.median(intervals, axis=0)
    uppers = np.quantile(intervals, 0.75, axis=0)
    lowers = np.quantile(intervals, 0.25, axis=0)
    middle = np.mean(midpoints)
    xerr0 = [middle - uppers[0], lowers[1] - middle]
    xerr1 = [middle - midpoints[0], midpoints[1] - middle]
    xerr2 = [middle - lowers[0], uppers[1] - middle]

    axs[1].errorbar(
        [middle],
        [i],
        xerr=np.array([xerr0]).T,
        fmt="o",
        color=colors[i],
        linewidth=2,
        capsize=4,
        markersize=0,
    )
    axs[1].errorbar(
        [middle],
        [i],
        xerr=np.array([xerr1]).T,
        fmt="o",
        color=colors[i],
        linewidth=2,
        capsize=4,
        markersize=0,
    )
    axs[1].errorbar(
        [middle],
        [i],
        xerr=np.array([xerr2]).T,
        fmt="o",
        color=colors[i],
        linewidth=2,
        capsize=4,
        markersize=0,
    )

axs[1].axvline(0.5, color="red", linestyle="--")
axs[1].set_ylim(-0.5, 1.5)
axs[1].set_yticks([])

axs[1].set_xlabel(r"90\% confidence intervals for $p$")
axs[1].set_xlim(0.45, 0.7)
plt.suptitle(r"Tests on $p=P(y_i,y_{i+1}\,$on same side of truth)")
plt.tight_layout()
plt.savefig("figs/pdg_indep.pdf")

plt.show()

In [ ]:
truth = truths["c"]
values = np.array(datasets["c"]["value"])
over = values > truth
xor = np.bitwise_xor(over[:-1], over[1:])
n_pairs = len(xor)
same_side = np.sum(~xor)
print(n_pairs, same_side, binom.cdf(k=same_side, n=n_pairs, p=0.5))

In [ ]:
import pymc as pm

ns = [len(datasets[n]) for n in particle_keys]
ks_mid = [np.sum(datasets[n]["value"] > truths[n][0]) for n in particle_keys]
ks_generous = []
for p in particle_keys:
    n = len(datasets[p])
    truth = truths[p][0]
    sigma = truths[p][1]
    contenders = [truth, truth - sigma, truth + sigma]
    k_contenders = np.array(
        [np.sum(datasets[p]["value"] > contender) for contender in contenders]
    )
    closest_to_half = np.argmin(np.abs(k_contenders / n - 0.5))
    ks_generous.append(k_contenders[closest_to_half])
rhos = []
for ks in [ks_mid, ks_generous]:
    with pm.Model() as model:
        rho = pm.Uniform("rho", lower=0, upper=1)
        alphabeta = (1 / rho) - 1
        alpha = alphabeta / 2
        beta = alphabeta / 2
        k = pm.BetaBinomial("k", alpha=alpha, beta=beta, n=ns, observed=ks)
        trace = pm.sample(20000, tune=1000)
    rhos.append(trace.posterior["rho"].values.flatten())

In [ ]:
plt.figure(figsize=(5, 2))
bins = np.linspace(0, 1, 401)
plt.hist(
    rhos[0],
    color="grey",
    density=True,
    bins=bins,
    histtype="step",
    linewidth=1,
    label=r"$\theta$ as truth",
)
plt.hist(
    rhos[1],
    color="black",
    density=True,
    bins=bins,
    histtype="step",
    linewidth=1,
    label=r"$\theta$, $\theta-\sigma$, or $\theta+\sigma$ as truth",
)
plt.xlim(0, 0.5)
plt.xlabel(r"$\rho$")
plt.ylabel("Posterior density")
plt.legend(frameon=False)
plt.savefig("figs/pdg_rho.pdf", bbox_inches="tight")
plt.show()

# Coverage

In [ ]:
# estimate cdfs for correlated binomial

binom_corr_cdfs = []
p = 0.55
for n in range(2, 11):
    binom_corr_pmf = np.zeros(n + 1)
    for i in range(100000):
        same_side = np.random.choice([-1, 1], size=n - 1, p=[1 - p, p])
        initial = np.random.choice([-1, 1])
        seq = np.concatenate(([initial], np.cumprod(same_side) * initial))
        seq = (seq + 1) // 2
        binom_corr_pmf[np.sum(seq)] += 1
    binom_corr_pmf /= np.sum(binom_corr_pmf)
    binom_corr_cdfs.append(np.cumsum(binom_corr_pmf))

In [ ]:
binom_corr_cdfs[1]

In [ ]:
plt.plot(binom_corr_cdfs[2])
plt.plot(binom.cdf(np.arange(5), n=4, p=0.5))
plt.show()

In [ ]:
# let's start with the particle dataset
from methods import (
    birge,
    random_effects_hksj,
    binomial_method,
    random_effects_dl,
    vniim,
    sign_rank,
    flip_interval,
    fixed_effect,
    linear_pool,
    birge_forecast,
    boot,
    random_effects_mle,
    random_effects_pm,
    interval_score,
)
from tqdm import tqdm
from collections import defaultdict
from math import comb
from itertools import combinations
from scipy.stats import norm, betabinom

# birge_covs = defaultdict(lambda: defaultdict(list))
# re_covs = defaultdict(lambda: defaultdict(list))
binom_target_covs = {}
signrank_target_covs = {}
target_covs = {}
binom_corr_target_covs = {}
# binom_covs = defaultdict(lambda: defaultdict(list))
# method_covs = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
n_combs = 100
methods = [
    "fe",
    "birge",
    "birge-t",
    "pdg",
    "codata",
    "re_hksj",
    "re_mmhksj",
    "re_dl",
    "re_mle",
    "re_pm",
    # 'vniim',
    "binom",
    "signrank",
    # 'boot-normal',
    # 'boot-student',
    "binom-corr",
    # 'birge-forecast'
    # 'flip',
    # "lp",
]
ns = np.arange(2, 11)
result_shape = (len(ns), len(methods), len(particle_keys), n_combs)
method_covs = np.full(result_shape, np.nan)
method_widths = np.full(result_shape, np.nan)
method_iscores = np.full(result_shape, np.nan)
method_widths_rel_fe = np.full(result_shape, np.nan)
# signrank_covs = defaultdict(lambda: defaultdict(list))

# target_cov = 1-2*binom_tail
# print(target_cov)
# target_cov = 0.75
# tail_prob = (1-target_cov)/2


def shrink_interval(interval, z_actual, z_target):
    width = interval[1] - interval[0]
    middle = (interval[0] + interval[1]) / 2
    new_width = width * z_target / z_actual
    return (middle - new_width / 2, middle + new_width / 2)


# ns = [6]
for i, n in enumerate(ns):
    base_target_cov = 0.6827
    if n > 2:
        _, binom_target_cov, _ = binomial_method(
            np.sort(np.arange(n)), coverage=base_target_cov
        )
        binom_target_covs[n] = binom_target_cov
        _, signrank_target_cov = sign_rank(
            np.sort(np.arange(n)), coverage=base_target_cov
        )
        signrank_target_covs[n] = signrank_target_cov
        # z_signrank = -norm.ppf((1-signrank_target_cov)/2)
        # _, binom_corr_target_cov = binomial_method(np.sort(np.arange(n)), cdf=binom_corr_cdfs[i], coverage=base_target_cov)
        # binom_corr_target_covs[n] = binom_corr_target_cov
        beta_binom_cdf = betabinom.cdf(np.arange(n + 1), n, a=4.5, b=4.5)
        _, binom_corr_target_cov, _ = binomial_method(
            np.sort(np.arange(n)), cdf=beta_binom_cdf, coverage=0.65
        )
        binom_corr_target_covs[n] = binom_corr_target_cov

        target_cov = base_target_cov

        z_binom = -norm.ppf((1 - binom_target_cov) / 2)
        z_binom_corr = -norm.ppf((1 - binom_corr_target_cov) / 2)

    else:
        target_cov = base_target_cov
    z_target_cov = -norm.ppf((1 - target_cov) / 2)
    target_covs[n] = target_cov

    for j, name in tqdm(enumerate(particle_keys)):
        df = datasets[name]
        if len(df) < n:
            continue
        value = np.array(df.value)
        sigma = np.array(df.uncertainty)
        truth_min = truths[name][0] - 3 * truths[name][1]
        truth_max = truths[name][0] + 3 * truths[name][1]
        truth = truths[name][0]

        # if the number of combinations of n results is small, we use all combinations,
        # otherwise we randomly sample
        if comb(len(value), n) <= n_combs:
            subset_idxs = [np.array(c) for c in combinations(range(len(value)), n)]
        else:
            subset_idxs = np.array(
                [
                    np.random.choice(len(value), size=n, replace=False)
                    for _ in range(n_combs)
                ]
            )
        for k, subset_idx in enumerate(subset_idxs):
            value_s = value[subset_idx]
            sigma_s = sigma[subset_idx]
            intervals = {}
            intervals["birge"], wm, _, _ = birge(value_s, sigma_s, coverage=target_cov)
            intervals["birge-t"], wm, _, _ = birge(
                value_s, sigma_s, coverage=target_cov, dist="t"
            )
            intervals["pdg"], wm, _, _ = birge(
                value_s, sigma_s, coverage=target_cov, pdg=True
            )
            intervals["codata"], wm, _, _ = birge(
                value_s, sigma_s, coverage=target_cov, codata=True
            )
            intervals["re_hksj"], muhat, _, _ = random_effects_hksj(
                value_s, sigma_s, coverage=target_cov
            )
            intervals["re_mmhksj"], muhat, _, _ = random_effects_hksj(
                value_s, sigma_s, coverage=target_cov, trunc="talpha"
            )
            intervals["re_dl"], muhat, _, _ = random_effects_dl(
                value_s, sigma_s, coverage=target_cov
            )
            intervals["re_pm"], muhat, _, _ = random_effects_pm(
                value_s, sigma_s, coverage=target_cov
            )
            intervals["re_mle"], muhat, _, _ = random_effects_mle(
                value_s, sigma_s, coverage=target_cov
            )
            # intervals['boot-normal'] = boot(value_s, sigma_s, coverage=target_cov, which='normal')
            # intervals['boot-student'] = boot(value_s, sigma_s, coverage=target_cov, which='studentized')
            # intervals['birge-forecast'], _, _, _, _ = birge_forecast(value_s, sigma_s, coverage=target_cov)
            # intervals['vniim'], wm = vniim(value_s, sigma_s, coverage=target_cov)
            if n > 2:
                interval, _, interval_shrink = binomial_method(
                    value_s, p=0.5, coverage=target_cov, shrink="cdf-interp"
                )
                intervals["binom"] = (
                    interval  # shrink_interval(interval, z_binom, z_target_cov)
                )
                interval, _, interval_shrink = binomial_method(
                    value_s, cdf=beta_binom_cdf, coverage=0.65, shrink="center"
                )
                intervals["binom-corr"] = (
                    interval  # shrink_interval(interval, z_binom_corr, z_target_cov)
                )
                interval, _ = sign_rank(value_s, coverage=target_cov)
                intervals["signrank"] = (
                    interval  # shrink_interval(interval, z_signrank, z_target_cov)
                )
                # fe intervals for different target coverages, for width comparison
                fe_binom_interval, _, _ = fixed_effect(
                    value_s, sigma_s, coverage=binom_target_cov
                )
                fe_binom_corr_interval, _, _ = fixed_effect(
                    value_s, sigma_s, coverage=binom_corr_target_cov
                )
                fe_signrank_interval, _, _ = fixed_effect(
                    value_s, sigma_s, coverage=signrank_target_cov
                )
            # intervals['flip'], _ = flip_interval(value_s, coverage=target_cov, max_iter=100, boot=False)
            intervals["fe"], _, _ = fixed_effect(value_s, sigma_s, coverage=target_cov)

            # intervals["lp"], _ = linear_pool(
            #    value_s, sigma_s, coverage=target_cov, gridn=1000
            # )

            for l, method in enumerate(methods):
                if method not in intervals:
                    continue
                interval = intervals[method]
                width = interval[1] - interval[0]
                if method == "binom":
                    method_target_cov = binom_target_cov
                    fe_width = fe_binom_interval[1] - fe_binom_interval[0]
                elif method == "binom-corr":
                    method_target_cov = binom_corr_target_cov
                    fe_width = fe_binom_corr_interval[1] - fe_binom_corr_interval[0]
                elif method == "signrank":
                    method_target_cov = signrank_target_cov
                    fe_width = fe_signrank_interval[1] - fe_signrank_interval[0]
                else:
                    method_target_cov = target_cov
                    fe_width = intervals["fe"][1] - intervals["fe"][0]
                method_widths_rel_fe[i, l, j, k] = width / fe_width
                method_covs[i, l, j, k] = (interval[0] <= truth_max) & (
                    truth_min <= interval[1]
                )
                method_widths[i, l, j, k] = interval[1] - interval[0]
                method_iscores[i, l, j, k] = interval_score(
                    interval, truth, coverage=method_target_cov, percent=True
                )
method_covs = np.nanmean(method_covs, axis=3)
method_widths = np.nanmean(method_widths, axis=3)
method_iscores = np.nanmean(method_iscores, axis=3)
method_widths_rel_fe = np.nanmean(method_widths_rel_fe, axis=3)

In [ ]:
print(method_covs.shape)
n_datasets = np.sum(~np.isnan(method_covs[:, 0, :]), axis=1)
print(n_datasets)

In [ ]:
colors = {
    "binom": "orange",
    "binom-corr": "goldenrod",
    "signrank": "red",
    "flip": "yellow",
    "fe": "grey",
    "lp": "lightgrey",
    "re_hksj": "green",
    "re_dl": "lightgreen",
    "vniim": "lightblue",
    "birge": "blue",
    "pdg": "lightblue",
    "codata": "plum",
    "boot-normal": "brown",
    "boot-student": "tan",
    # 'birge-forecast': 'pink'
}

method_widths_rel = method_widths / method_widths[:, 0, np.newaxis, :]

# bootstrap over datasets
B = 400
method_covs_boot = np.full((len(ns), len(methods), len(particle_keys), B), np.nan)
for b in range(B):
    dataset_idxs = np.random.choice(
        len(particle_keys), size=len(particle_keys), replace=True
    )
    method_covs_boot[:, :, :, b] = method_covs[:, :, dataset_idxs]
method_covs_mean = np.nanmean(method_covs, axis=2)
method_covs_mean_rel = (
    method_covs_mean / method_covs_mean[:, methods.index("signrank"), np.newaxis]
)
method_covs_mean_boot = np.nanmean(method_covs_boot, axis=2)
method_covs_mean_boot_rel = (
    method_covs_mean_boot
    / method_covs_mean_boot[:, methods.index("signrank"), np.newaxis, :]
)

uppers_rel = 2 * method_covs_mean_rel - np.nanquantile(
    method_covs_mean_boot_rel, 0.1, axis=2
)
lowers_rel = 2 * method_covs_mean_rel - np.nanquantile(
    method_covs_mean_boot_rel, 0.9, axis=2
)

In [ ]:
method_covs_mean_rel[1]

In [ ]:
# hspace = 0
fig, axs = plt.subplots(3, 1, figsize=(8, 7), sharex=True, gridspec_kw={"hspace": 0})
for i, method in enumerate(methods):
    axs[0].plot(
        ns, np.nanmean(method_covs[:, i, :], axis=1), label=method, color=colors[method]
    )
    axs[1].plot(
        ns,
        np.nanmean(method_widths_rel[:, i, :], axis=1),
        label=method,
        color=colors[method],
    )
    axs[2].plot(ns, method_covs_mean_rel[:, i], label=method, color=colors[method])
    axs[2].plot(
        ns, uppers_rel[:, i], color=colors[method], linestyle="--", linewidth=0.5
    )
    axs[2].plot(
        ns, lowers_rel[:, i], color=colors[method], linestyle="--", linewidth=0.5
    )
# axs[0].plot(signrank_target_covs.keys(), signrank_target_covs.values(), label='target (signrank)', color='grey', linestyle='--')
# axs[0].plot(binom_target_covs.keys(), binom_target_covs.values(), label='target (binom)', color='grey', linestyle=':')
# axs[0].plot(binom_corr_target_covs.keys(), binom_corr_target_covs.values(), label='target (binom-corr)', color='grey', linestyle='--')
axs[0].plot(
    target_covs.keys(),
    target_covs.values(),
    label="target",
    color="black",
    linestyle="--",
    linewidth=2,
)
axs[0].set_ylabel("Coverage")
# axs[0].legend()
axs[1].set_ylabel("Width relative to fixed effect")
axs[2].legend(frameon=False)

axs[2].set_xlabel("Number of results")
axs[2].set_ylabel("Coverage relative to signrank")

# point ticks inwards and add top and right ticks
for ax in axs:
    ax.tick_params(direction="in", top=True, right=True)

plt.show()

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(8, 3.5), sharex=True, gridspec_kw={"hspace": 0})
linewidth = 1.5

for i, result in enumerate([method_covs, method_widths_rel_fe]):
    axs[i].plot(
        ns,
        np.nanmean(result[:, methods.index("re_dl"), :], axis=1),
        label="RE (DL)",
        color="black",
        linewidth=linewidth,
    )
    axs[i].plot(
        ns,
        np.nanmean(result[:, methods.index("re_hksj"), :], axis=1),
        label="RE (HKSJ)",
        color="black",
        linewidth=linewidth,
        linestyle=":",
    )
    axs[i].plot(
        ns,
        np.nanmean(result[:, methods.index("re_mmhksj"), :], axis=1),
        label="RE (mmHKSJ)",
        color="black",
        linewidth=linewidth,
        linestyle=(0, (1, 8)),
    )
    axs[i].plot(
        ns,
        np.nanmean(result[:, methods.index("re_mle"), :], axis=1),
        label="RE (MLE)",
        color="black",
        linewidth=linewidth,
        linestyle="--",
    )
    axs[i].plot(
        ns,
        np.nanmean(result[:, methods.index("re_pm"), :], axis=1),
        label="RE (PM)",
        color="black",
        linewidth=linewidth,
        linestyle="-.",
    )
    axs[i].plot(
        ns,
        np.nanmean(result[:, methods.index("fe"), :], axis=1),
        label="Fixed Effect",
        color="grey",
        linewidth=linewidth,
    )

axs[0].set_ylabel("Coverage")
axs[0].plot(
    target_covs.keys(), target_covs.values(), color="black", linestyle="-", linewidth=1
)
axs[0].text(
    9,
    list(target_covs.values())[-1] * 0.98,
    r"$\uparrow$ Target coverage",
    va="top",
    ha="center",
)
axs[0].text(8, 0.45, r"Achieved coverage", va="bottom", ha="center")

plt.xlabel("Number of results $n$")
axs[1].set_ylabel(r"Average width\\relative to Fixed Effect")

axs[0].legend(
    frameon=False,
    ncol=6,
    prop={"size": 8},
    bbox_to_anchor=(0.0, 1.02, 1.0, 0.102),
    loc="lower left",
    mode="expand",
    borderaxespad=0.0,
)

plt.xlim(2, 10)

for ax in axs:
    ax.tick_params(direction="in", top=True, right=True)
plt.savefig("figs/pdg1970_cov_re.pdf", bbox_inches="tight")
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(8, 3.5), sharex=True, gridspec_kw={"hspace": 0})
linewidth = 1.5

for i, result in enumerate([method_covs, method_widths_rel_fe]):
    axs[i].plot(
        ns,
        np.nanmean(result[:, methods.index("pdg"), :], axis=1),
        label="PDG",
        color="black",
        linewidth=linewidth,
        linestyle="-",
    )
    axs[i].plot(
        ns,
        np.nanmean(result[:, methods.index("re_pm"), :], axis=1),
        label="RE (PM)",
        color="black",
        linewidth=linewidth,
        linestyle="--",
    )
    axs[i].plot(
        ns,
        np.nanmean(result[:, methods.index("fe"), :], axis=1),
        label="Fixed Effect",
        color="grey",
        linewidth=linewidth,
    )

axs[0].set_ylabel("Coverage")
axs[0].plot(
    target_covs.keys(), target_covs.values(), color="black", linestyle="-", linewidth=1
)
axs[0].text(
    9,
    list(target_covs.values())[-1] * 0.98,
    r"$\uparrow$ Target coverage",
    va="top",
    ha="center",
)
# axs[0].text(8, 0.45, r'Achieved coverage', va='bottom', ha='center')

plt.xlabel("Number of results $n$")
axs[1].set_ylabel(r"Average width\\relative to Fixed Effect")

axs[0].legend(
    frameon=False,
    ncol=6,
    prop={"size": 8},
    bbox_to_anchor=(0.0, 1.02, 1.0, 0.102),
    loc="lower left",
    mode="expand",
    borderaxespad=0.0,
)

plt.xlim(2, 10)

for ax in axs:
    ax.tick_params(direction="in", top=True, right=True)
plt.savefig("figs/pdg1970_cov_re-pdg.pdf", bbox_inches="tight")
plt.savefig("figs/pdg1970_cov_re-pdg.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
np.nanmean(method_covs[:, methods.index("re_hksj"), :], axis=1)

In [ ]:
random_effects_hksj(value, sigma, coverage=0.6827, trunc=True)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(8, 3.5), sharex=True, gridspec_kw={"hspace": 0})
linewidth = 1.5

for i, result in enumerate([method_covs, method_widths_rel_fe]):
    axs[i].plot(
        ns,
        np.nanmean(result[:, methods.index("birge"), :], axis=1),
        label="Birge Ratio",
        color="black",
        linewidth=linewidth,
    )
    axs[i].plot(
        ns,
        np.nanmean(result[:, methods.index("birge-t"), :], axis=1),
        label="Birge Ratio ($t$)",
        color="black",
        linewidth=linewidth,
        linestyle="-.",
    )
    axs[i].plot(
        ns,
        np.nanmean(result[:, methods.index("pdg"), :], axis=1),
        label="PDG",
        color="black",
        linewidth=linewidth,
        linestyle=":",
    )
    axs[i].plot(
        ns,
        np.nanmean(result[:, methods.index("codata"), :], axis=1),
        label="CODATA",
        color="black",
        linewidth=linewidth,
        linestyle="--",
    )
    axs[i].plot(
        ns,
        np.nanmean(result[:, methods.index("fe"), :], axis=1),
        label="Fixed Effect",
        color="grey",
        linewidth=linewidth,
    )

axs[0].set_ylabel("Coverage")
axs[0].plot(
    target_covs.keys(), target_covs.values(), color="black", linestyle="-", linewidth=1
)
axs[0].text(
    9,
    list(target_covs.values())[-1] * 0.98,
    r"$\uparrow$ Target coverage",
    va="top",
    ha="center",
)
axs[0].text(8, 0.49, r"Achieved coverage", va="bottom", ha="center")
axs[0].legend(frameon=False)

plt.xlabel("Number of results $n$")
axs[1].set_ylabel(r"Average width\\relative to Fixed Effect")

axs[0].legend(
    frameon=False,
    ncol=5,
    prop={"size": 8},
    bbox_to_anchor=(0.0, 1.02, 1.0, 0.102),
    loc="lower left",
    mode="expand",
    borderaxespad=0.0,
)

plt.xlim(2, 10)

for ax in axs:
    ax.tick_params(direction="in", top=True, right=True)
plt.savefig("figs/pdg1970_cov_birge.pdf", bbox_inches="tight")
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(8, 5.6), sharex=True, gridspec_kw={"hspace": 0})

nonparam_target_covs = {
    'binom': binom_target_covs,
    'binom-corr': binom_corr_target_covs,
    'signrank': signrank_target_covs,
}
colors = {
    'binom': 'black',
    'binom-corr': 'red',
    'signrank': 'silver',
}
labels = {
    'binom': 'ST',
    'binom-corr': r"ST$_{\rho=0.1}$",
    'signrank': r"SRT",
}
linewidths = defaultdict(lambda: 3)
linewidths['binom-corr'] = 1.5

axs[0].set_ylabel("Coverage")
axs[1].set_ylabel("Width relative to FE")
plt.xlabel("Number of results $n$")

# plt.plot(target_covs.keys(), target_covs.values(), label='target', color='black', linestyle='--', linewidth=2)
# print(binom_corr_target_covs[3])
plt.xlim(3, 10)
axs[1].set_ylim(1.5, 4.2)
axs[0].tick_params(direction="in", top=True, right=True)
axs[1].tick_params(direction="in", top=True, right=True)
for method in ['binom', 'signrank', 'binom-corr']:
    for i, result in enumerate([method_covs, method_widths_rel_fe]):
        axs[i].plot(
            ns,
            np.nanmean(result[:, methods.index(method), :], axis=1),
            label=labels[method],
            color=colors[method],
            linewidth=linewidths[method],
        )
    axs[0].plot(
        nonparam_target_covs[method].keys(),
        nonparam_target_covs[method].values(),
        color=colors[method],
        linestyle="--",
        linewidth=1.5,
    )
    plt.legend(frameon=False)
    if method == 'signrank':
        plt.savefig("figs/pdg1970_cov_nonparam_less.png", dpi=300, bbox_inches="tight")

plt.savefig("figs/pdg1970_cov_nonparam.pdf", bbox_inches="tight")
plt.savefig("figs/pdg1970_cov_nonparam.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(8, 5.6), sharex=True, gridspec_kw={"hspace": 0})

nonparam_target_covs = {
    'binom': binom_target_covs,
    'binom-corr': binom_corr_target_covs,
    'signrank': signrank_target_covs,
}
colors = {
    're_pm': 'blue',
    'pdg': 'green',
    'fe': 'grey',
    'binom': 'black',
    'binom-corr': 'red',
    'signrank': 'red',
}
labels = {
    'binom': 'ST',
    'binom-corr': r"ST$_{\rho=0.1}$",
    'signrank': r"SRT",
    're_pm': r"Random Effects (PM)",
    'pdg': r"Birge (PDG)",
    'fe': r"Fixed Effect",
}
linewidths = defaultdict(lambda: 3)
linewidths['binom-corr'] = 1.5

axs[0].set_ylabel("Coverage")
axs[1].set_ylabel("Coverage")
plt.xlabel("Number $n$ of studies in meta-analysis")

# plt.plot(target_covs.keys(), target_covs.values(), label='target', color='black', linestyle='--', linewidth=2)
# print(binom_corr_target_covs[3])
plt.xlim(2, 10)
# axs[1].set_ylim(1.5, 4.2)
for ax in axs:
    ax.tick_params(direction="in", top=True, right=True)
    ax.set_ylim(0.3, 0.95)
which_ax = {
    'fe': 0,
    're_pm': 0,
    'pdg': 0,
    'binom': 1,
    'signrank': 1,
    'binom-corr': 1,
}
axs[0].axhline(0.6827, color='black', linestyle='--', linewidth=1.5)
# axs[0].text(
#     7.5,
#     0.6827+0.01,
#     r"Target coverage",
#     va="bottom",
#     ha="center",
# )
for method in ['fe', 're_pm', 'pdg', 'binom', 'signrank']:
    ax = axs[which_ax[method]]
    ax.plot(ns, np.nanmean(method_covs[:, methods.index(method), :], axis=1),
            label=labels[method],
            color=colors[method],
            linewidth=linewidths[method],
        )
    if method in ['binom', 'signrank', 'binom-corr']:
        ax.plot(
            nonparam_target_covs[method].keys(),
            nonparam_target_covs[method].values(),
            color=colors[method],
            linestyle="--",
            linewidth=1.5,
        )

axs[0].legend(frameon=False, loc='upper left')
axs[1].legend(frameon=False, loc='lower left')

# plt.savefig("figs/pdg1970_cov_nonparam.pdf", bbox_inches="tight")
plt.savefig("figs/pdg1970_cov_poster.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

alpha = 1 / 3
sigma1 = np.linspace(0.1, 2.5, 1000)
zalpha = norm.ppf(1 - alpha / 2)
scores = 2 * sigma1 * zalpha + (4 / alpha) * (
    norm.pdf(sigma1 * zalpha) / norm.cdf(-sigma1 * zalpha) - sigma1 * zalpha
) * norm.cdf(-sigma1 * zalpha)
covs = 1 - 2 * norm.cdf(-sigma1 * zalpha)
plt.plot(sigma1, scores)
plt.plot(sigma1, covs)
plt.axvline(sigma1[np.argmin(scores)])

In [ ]:
# bootstrap over datasets
plt.figure(figsize=(10, 5))

method_covs_boot_mean = np.nanmean(method_covs_ds_boot, axis=2)
method_covs_mean = np.nanmean(method_covs, axis=(2, 3))
uppers = 2 * method_covs_mean - np.nanquantile(method_covs_boot_mean, 0.1, axis=2)
lowers = 2 * method_covs_mean - np.nanquantile(method_covs_boot_mean, 0.9, axis=2)
for i, method in enumerate(methods):
    plt.plot(
        ns,
        np.nanmean(method_covs[:, i, :, :], axis=(1, 2)),
        label=method,
        color=colors[method],
    )
    plt.plot(ns, uppers[:, i], color=colors[method], linestyle=":", linewidth=0.5)
    plt.plot(ns, lowers[:, i], color=colors[method], linestyle=":", linewidth=0.5)
plt.plot(ns, target_covs, color="black", linestyle="--", linewidth=2)
plt.legend()
plt.show()

In [ ]:
n_datasets

In [ ]:
truths

In [ ]:
y = np.array(datasets["agi"].value)
sigma = np.array(datasets["agi"].uncertainty)

import pymc as pm

print(y)
n = len(y)
width = np.max(y) - np.min(y)

with pm.Model() as model:
    theta = pm.Normal("theta", np.mean(y), np.mean(y))
    # scaler_rate = pm.Exponential('scaler_rate', 0.2)
    # scalers = pm.Exponential('scalers', scaler_rate, shape=n)+1
    # y_pred = pm.Normal('y_pred', theta, sigma*scalers, observed=y)

    # adder_rate = pm.Exponential('adder_rate', 10/width)
    adders = pm.Exponential("adders", 5 / width, shape=n)
    y_pred = pm.Normal("y_pred", theta, np.sqrt(sigma**2 + adders**2), observed=y)

    trace = pm.sample(draws=1000, tune=1000, chains=4, target_accept=0.99)

In [ ]:
pm.plot_trace(trace)

In [ ]:
for i in range(n):
    plt.hist(
        trace.posterior["scalers"].values[:, :, i].flatten() + 1,
        bins=100,
        histtype="step",
    )
plt.show()

In [ ]:
trace.posterior["scalers"].values.shape

In [ ]:
len(y)

In [ ]:
plt.hist(trace.posterior["theta"].values.flatten(), bins=100)
qs = np.quantile(trace.posterior["theta"].values.flatten(), [0.16, 0.84])
plt.axvline(qs[0], color="red", linestyle="--", linewidth=1)
plt.axvline(qs[1], color="red", linestyle="--", linewidth=1)
ymax = plt.ylim()[1]
if "scalers" in trace.posterior:
    scaler_qs = (
        np.quantile(trace.posterior["scalers"].values, [0.25, 0.5, 0.75], axis=(0, 1))
        + 1
    )
    print(scaler_qs.shape)
    for i in range(3):
        plt.errorbar(
            y,
            np.linspace(ymax / len(y), ymax, len(y)),
            xerr=sigma * scaler_qs[i],
            fmt=".",
            markersize=2,
            linewidth=1,
            color="black",
            capsize=2,
        )
if "adders" in trace.posterior:
    adder_qs = np.quantile(
        trace.posterior["adders"].values, [0.25, 0.5, 0.75], axis=(0, 1)
    )
    print(adder_qs.shape)
    for i in range(3):
        plt.errorbar(
            y,
            np.linspace(ymax / len(y), ymax, len(y)),
            xerr=sigma + adder_qs[i],
            fmt=".",
            markersize=2,
            linewidth=1,
            color="black",
            capsize=2,
        )
# sigmas_adjust = sigma * np.median(trace.posterior['scalers'].values, axis=(0,1))

plt.errorbar(
    y,
    np.linspace(ymax / len(y), ymax, len(y)),
    xerr=sigma,
    fmt=".",
    markersize=2,
    linewidth=2,
    color="black",
)

# plt.ylim(0, ymax)
# plt.xlim(qs[0], qs[1])
plt.show()